In [54]:
import numpy as np
from collections import defaultdict

In [85]:
class MatrixMetersCoordinates:
    def __init__(self,
        width: int = 200000,
        height: int = 100000,
        step: int=100,
        longitude_west: float=5.732,
        latitude_north:float=47.747,
    ):
        self.width = width
        self.height = height
        self.step = step
        self.longitude_west = longitude_west
        self.latitude_north = latitude_north
        
        self.post_init()
    
    def post_init(self):
        
        matrix_meters = np.zeros((int(self.width/self.step), int(self.height/self.step)))
        self.matrix_meters = matrix_meters
        
        matrix_meters_metadata = {"step":self.step}
        self.matrix_meters_metadata = matrix_meters_metadata
        
        self.get_coordinates_meters_hashes()
        
    def get_coordinates_meters_hashes(self):
        earth_radius = 6371000
        longitude_est = self.longitude_west + (self.width/earth_radius) * (180/ np.pi) / np.cos(self.latitude_north * np.pi/180)
        latitude_south = self.latitude_north - (self.height/earth_radius) * (180/ np.pi)
        
        longitude_west_to_est = np.linspace(self.longitude_west, longitude_est, self.matrix_meters.shape[0])
        latitude_north_to_south = np.linspace(self.latitude_north, latitude_south, self.matrix_meters.shape[1])
        hash_coordinates_to_indices = dict()

        for ix in range(len(longitude_west_to_est)-1):
            for iy in range(len(latitude_north_to_south)-1):
                hash_coordinates_to_indices[
                    (
                        longitude_west_to_est[ix],
                        latitude_north_to_south[iy],
                        longitude_west_to_est[ix+1],
                        latitude_north_to_south[iy+1]
                    )
                ] = (ix, iy)
        self.hash_coordinates_to_indices = hash_coordinates_to_indices
        self.hash_indices_to_coordinates = {value: key for key, value in self.hash_coordinates_to_indices.items()}
                
def to_geojson(decibels_matrix, hash_indices_to_coordinates, filepath):
    #https://stackoverflow.com/questions/56683507/different-color-for-each-polygon-in-multipolygon-in-mapbox
    for x in range(len(decibels_matrix.shape[0])):
        for y in range(len(decibels_matrix.shape[1])):
            coordinates = hash_indices_to_coordinates[(x,y)]
            decibels = decibels_matrix[x,y]
            
            